# Capstone Project - Ice cream shops in Warsaw

## Introduction / Business Problem

As average temperatures rise on an yearly basis in Warsaw, selling ice cream is the type of business that can bring significant profits, especially in the heated summer months. Natural ice cream (or "lody naturalne", as locals call it) is becoming more and more popular and entreprenuers with very well crafted recipes and marketing strategies are asking "where should I open my shop?", as the location of the shop could exponentially impact the returns.

Through this project, I intend to help such entrepreneurs make the right decision when it comes to choosing the location to open their shop using unsupervised learning techniques. 

## Data

The data used to solve this problem is geospatial data collected from FourSquare. Our data is a single dataframe, containing location data (lat, lng), where lat stands for latitude and lng for longitude. The name of the shop is also collected, but it is not absolutely necessary for the analysis. We will use kernel density estimation (KDE) to determine the area of influence of the existing ice cream shops and then provide suggestions of places to open the new ice cream shop outside this area

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

import pprint

print('Libraries imported.')

Libraries imported.


In [5]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

credfile.close()

TypeError: 'list' object cannot be interpreted as an integer

In [35]:
LIMIT = 300 # limit of number of venues returned by Foursquare API

radius = 10000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    52.237049, 
    21.017532, 
    radius, 
    LIMIT,
    '4bf58dd8d48988d1c9941735'
    )

In [39]:
results = requests.get(url).json()
#pprint.pprint(results)

In [37]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [38]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

C:\Users\Acer1\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Häagen-Dazs,Ice Cream Shop,52.233882,21.019364
1,Cukiernia Pawłowicz,Ice Cream Shop,52.232760,21.016609
2,Na Końcu Tęczy,Ice Cream Shop,52.219837,21.019050
3,Lodziarnia Limoni,Ice Cream Shop,52.237261,21.002209
4,N'Ice Cream Factory,Ice Cream Shop,52.232856,21.017677
5,Lody Prawdziwe,Ice Cream Shop,52.235187,21.006428
6,Jednorożec Lody tradycyjne,Ice Cream Shop,52.205944,21.013504
7,Lodziarnia Ulica Baśniowa,Ice Cream Shop,52.262135,20.980712
8,Lukullus lody,Ice Cream Shop,52.224645,21.034609
9,Lodziarnia Malinova,Ice Cream Shop,52.203823,21.009882
